# Creación y Almacenamiento de Embeddings en ChromaDB

En el presente Notebook se crearán la documentación base del esquema reducido con el que trabajaremos, basada en el `information_schema` correspondiente, que luego se nutrirá por fuera con mayor información relevante para negocio. 

Adicionalmente, por fuera del Notebook, se confeccionará más documentación, como reglas de negocio y few-shot examples para, finalmente, obtener chunks de todos estos documentos y crear y almacenar los mismos, con su metadata asociada, en una base de datos vectorial de Chroma DB.

---

## Inicialización

### Librerías



In [1]:
# Importamos librerías estándar
import sys
from pathlib import Path
from dotenv import load_dotenv
import yaml
import json
from IPython.display import Markdown

from collections import OrderedDict
import pandas as pd

from langchain.prompts import ChatPromptTemplate
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain.schema import StrOutputParser


# Aplicamos operaciones sobre librerías estándar
load_dotenv('../.env')
pd.set_option('display.max_columns', None)
yaml.add_representer(OrderedDict, lambda dumper, data: dumper.represent_dict(data.items()))


# Añadimos el directorio raíz del proyecto al path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))


# Importamos paquetes personalizados
from src.utils.doc_generators import BaseMdlGenerator
from src.utils.splitters import MdlSplitter, ExtendedMarkdownSplitter, JsonExamplesSplitter
from src.back.embeddings import GenAIExtendedEmbeddingFunction
from src.back.chroma_collections import (
    MdlHierarchicalChromaCollections, 
    ContextEnricherChromaCollection, 
    ExamplesChromaCollection
)


### Constantes

In [2]:
TARGET_DB_NAME = 'adventure_works_dw'
TARGET_SCHEMA_NAME = 'sales'

BASE_QUERY_PATH = '../data/embeddings/auxs/get_information_schema.sql'
OUTPUT_PATH = '../data/embeddings/auxs'
MDL_PATH = '../data/embeddings/documents/MDL_adventure_works_dw.yaml'
BUSINESS_RULES_PATH = '../data/embeddings/documents/business_rules.md'
EXAMPLES_PATH = '../data/embeddings/documents/query_examples.json'

BUSINESS_RULES_CHUNK_SIZE = 2000
BUSINESS_RULES_CHUNK_OVERLAP = int(0.15 * BUSINESS_RULES_CHUNK_SIZE)

CHROMA_DB_PATH = '../data/embeddings/chroma'
TABLES_SUMMARY_COLLECTION_NAME = 'mdl_tables_summary'
COLUMNS_COLLECTION_NAME = 'mdl_columns'
BUSINESS_LOGIC_COLLECTION_NAME = 'business_logic'
EXAMPLES_COLLECTION_NAME = 'query_examples'

AZURE_OPENAI_EMBEDDING_MODEL = 'text-embedding-3-large'
GENAI_EMBEDDING_MODEL = 'gemini-embedding-001'

---

## Fichero YAML con MDL del esquema de interés

### Obtener fichero base desde `information_schema`

Primero obtendremos un fichero base construido utilizando la query `/data/embeddings/auxs/get_information_schema.sql`, identificada bajo la constante `BASE_QUERY_PATH`, sobre el que luego se añadirá metadata extra. Para esto, previamente, en la sección **Clases personalizadas**, hemos implementado la clase `BaseMdlGenerator`, que contine métodos que nos facilitarán estas tareas.

Mostremos los resultados en Pandas para que sean más vistosos:

In [3]:
mdl_generator = BaseMdlGenerator(BASE_QUERY_PATH)
information_schema_data = mdl_generator.get_information_schema_data(
    target_db_names= TARGET_DB_NAME,
    target_schema_names= TARGET_SCHEMA_NAME,
    format= 'pandas'
)

information_schema_data

,db_name,schema_name,table_name,column_name,column_type,primary_key,foreign_key,target
0,adventure_works_dw,sales,dim_customer,customer_key,INT4,True,False,None
1,adventure_works_dw,sales,dim_customer,geography_key,INT4,False,True,sales.dim_geography.geography_key
2,adventure_works_dw,sales,dim_customer,customer_full_name,TEXT,False,False,None
3,adventure_works_dw,sales,dim_customer,birth_date,DATE,False,False,None
4,adventure_works_dw,sales,dim_customer,marital_status,BPCHAR(1),False,False,None
...,...,...,...,...,...,...,...,...
107,adventure_works_dw,sales,fact_sales,freight,NUMERIC,False,False,None
108,adventure_works_dw,sales,fact_sales,order_date,DATE,False,False,None
109,adventure_works_dw,sales,fact_sales,due_date,DATE,False,False,None
110,adventure_works_dw,sales,fact_sales,ship_date,DATE,False,False,None


Ahora procederemos a crear el YAML base con el MDL de nuestro esquema, que podremos tomar como punto de partida para luego añadirle metadata extra manualmente:

In [4]:
mdl_generator.create_base_mdl_files(
    OUTPUT_PATH,
    TARGET_DB_NAME,
    TARGET_SCHEMA_NAME,
    encoding= 'utf-8'
)

['../data/embeddings/auxs/MDL_adventure_works_dw.yaml']

### Creación de embeddings

#### Chunking

Tomando como base el fichero obtenido en el apartado anterior, se ha creado un fichero que del MDL de nuestro esquema de interés que ha sido nutrido con metadata adicional, como descripciones para cada tabla y campo. Este fichero se encuentra en la ruta `/data/embeddings/documents/MDL_adventure_works_dw.yaml`.

Procederemos ahora a procesar este fichero para crear chunks de la información de cada una de las tablas y almacenar sus embeddings en nuestra base de datos vectorial de `chroma_db`, persistida en la ruta `/data/embeddings/chroma_db/`.

En la sección **Clases personalizadas**, hemos implementado la clase `MDLSplitter`, que nos permitirá obtener tanto los chunks correspondientes a los summary de las tablas, como también el de cada una de las columnas que no sean claves dentro de cada tabla:

In [5]:
mdl_splitter = MdlSplitter()
table_chunks, column_chunks = mdl_splitter.split_documents(MDL_PATH, 'utf-8')

Veamos ahora el aspecto que tiene, por ejemplo, el chunk de mayor tamaño para cada uno de los tipos:

In [6]:
max_len = max([len(c.page_content) for c in table_chunks])
print(f'{max_len=}')
print()
print([c.page_content for c in table_chunks if len(c.page_content) == max_len][0])
print()
print([c.metadata for c in table_chunks if len(c.page_content) == max_len][0])

max_len=1020

Database: adventure_works_dw
Schema: sales
Table: fact_sales
Table description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.
Table PRIMARY KEY:
- sales_order_number
- sales_order_line_number
Table FOREIGN KEYS (Column name, Reference):
- (product_key, sales.dim_product.product_key)
- (reseller_key, sales.dim_reseller.reseller_key)
- (employee_key, sales.dim_sales_person.employee_key)
- (customer_key, sales.dim_customer.customer_key)
- (promotion_key, sales.dim_promotion.promotion_key)
- (sales_territory_key, sales.dim_sa

In [7]:
max_len = max([len(c.page_content) for c in column_chunks])
print(f'{max_len=}')
print()
print([c.page_content for c in column_chunks if len(c.page_content) == max_len][0])
print()
print([c.metadata for c in column_chunks if len(c.page_content) == max_len][0])

max_len=271

Column name: spanish_product_subcategory_name
Column data type: VARCHAR(50)
Column description: Sub-categoría a la que pertenece el producto (ESPAÑOL). Cada categoría puede tener múltiples sub-categorías, llegando a tener hasta 150 valores diferentes. NUNCA viene a NULL.

{'file_name': 'MDL_adventure_works_dw.yaml', 'table_id': '5f915455-cb64-45ff-8662-59b92e5b79a7', 'database_name': 'adventure_works_dw', 'schema_name': 'sales', 'table_name': 'dim_product', 'column_name': 'spanish_product_subcategory_name', 'column_data_type': 'VARCHAR(50)'}


#### LangChain: ChromaDB + Azure OpenAI Embeddings

Ahora haremos una primera implementación de prueba de Chroma valiéndonos de funcionalidades de la librería LangChain, usando un modelo de embeddings de OpenAI a través de Azure AI Foundry.

El primer paso es crear un cliente persistente de ChromaDB, e inicializar las colecciones tanto a nivel de tablas como de columnas. Al inicializar las colecciones, especificaremos que queremos usar como métrica de distancia la distancia de Coseno, que se calcula de la siguiente manera:

$$Distancia = 1 - \frac{A \cdot B}{\|A\| \|B\|}$$

Donde:
* $A \cdot B$ es el producto punto (producto escalar) de los vectores $A$ y $B$.
* $∥A∥$ y $∥B∥$ son las magnitudes (o normas euclidianas) de los vectores $A$ y $B$, respectivamente.
​

La métrica **Distancia del Coseno** mide solo el ángulo entre vectores, lo que lo hace ideal para embeddings de texto, y particularmente para modelos de embeddings como [`text-embedding-3-large` de OpenAI cuyos valores vienen normalizados](https://help.openai.com/en/articles/8984345-which-distance-function-should-i-use).

Como es fácil ver en su expresión, esta métrica de distancia devuelve valores en el rango $[0, 2]$. Cuanto más cerca de 0, más similares serán los vectores $A$ y $B$. Esto nos deja en la necesidad de definir una métrica de `relevance_score` personalizada, que nos permita reescalar los resultados en valores comprendidos entre $[0, 1]$, siendo los valores más cercanos a 1 indicativo de mayor similitud entre los vectores. Esta métrica la hemos definido en la función `cosine_distance_relevance_score_fn` de la siguiente manera:

$$R_{score} = 1 - \frac{Distancia}{2}$$



In [8]:
openai_embeddings = AzureOpenAIEmbeddings(model=AZURE_OPENAI_EMBEDDING_MODEL)

mdl_hierarchical_openai_collection = MdlHierarchicalChromaCollections(
    collection_names= (TABLES_SUMMARY_COLLECTION_NAME, COLUMNS_COLLECTION_NAME),
    create_collections_if_not_exists= True,
    clear_collections= True,
    embedding_function= openai_embeddings,
    collections_configuration= {'hnsw': {'space': 'cosine'}}
)

mdl_hierarchical_openai_collection.add_documents(documents= (table_chunks, column_chunks))

(['427a1063-be9c-4cbf-a799-6bbc299e311f',
  'ec141c68-7f70-4798-b535-89f210517b27',
  '5f915455-cb64-45ff-8662-59b92e5b79a7',
  '3ded2fe5-7a89-4e82-86ca-9f677c13bd80',
  'df28f29d-594b-485e-91e8-84f0ed2405e4',
  '36845231-b072-4bcb-9f19-ef298119e2e2',
  'cc03e2fd-d7c4-42c5-bd6c-aa05977ff1e5',
  '61d35dc6-5611-43c7-980f-ce1d3ca443be',
  'f1053ef3-cef7-4ed9-83f6-912b4f022b1d',
  '6653bb58-4980-405e-9054-4e454af25fdf'],
 ['423bb90a-12b8-494a-8820-5650bfb2214b',
  'c28acb2d-1cdb-4a87-ac0f-1a00f0495c18',
  'cabbb2d5-aba6-40e4-906b-8c9fc54e7b77',
  '33df9c66-f890-48de-94d0-c93cb59fce52',
  'cc8e42d6-adcc-4ef2-9d46-16c625cb9c7f',
  '31e42ce5-4c35-410b-ade7-3940ab34efd7',
  '5ff31561-ac7c-4a5c-9336-a2d1e40a86a7',
  '88a18721-4b33-40a9-89df-a576eca8e921',
  '90127f9b-ace2-4a10-8be7-71ee1913edcb',
  'bf98eb49-e76f-4991-8e4c-ef5f265ae568',
  '60950c74-579f-4b6c-a2b5-1955edc68e2b',
  'f86ae0a9-9368-48a7-b40b-d7c47d21bcdb',
  'a586252e-8075-451e-b2b2-2e064815402b',
  '1373b85b-3712-4adf-b09b-0caf4a

##### Hierarchical RAG

Dada la naturaleza de nuestras colecciones, donde las columnas existen dentro del contexto de cada tabla, nos proponemos a utilizar la técnica de [**RAG jerárquico**](https://github.com/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/hierarchical_indices.ipynb) sobre las coleciones creadas. Esto significa que, primero buscaremos las tablas que sean más relevantes semánticamente para la query del usuario (en la coleción `tables_summary`) y, luego, para cada tabla obtenida, buscaremos sus columnas más relevantes (en la coleción `collumns`, filtrando la metada para que coincida con el dato de la tabla):

In [9]:
QUERY = 'Los 10 artículos más comprados.'

openai_relevant_docs = mdl_hierarchical_openai_collection.hierarchical_similarity_search(
    queries= QUERY,
    k_tables= 10,
    tables_score_threshold= 0.60,
    k_columns= 15,
    columns_score_threshold= 0.60
)

print(json.dumps(openai_relevant_docs, indent=2, ensure_ascii=False))

No relevant docs were retrieved using the relevance score threshold 0.6


[
  [
    {
      "table_summary": {
        "content": "Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promot

In [10]:
print(openai_relevant_docs)

[[{'table_summary': {'content': 'Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promotion.promotion_key)\n- (s

In [11]:
llm = AzureChatOpenAI(model='gpt-4o-mini', temperature=0)

SYSTEM_PROMPT = """
You are a context summarizer that receives retrieval documents in JSON format.
Your role is to:

1. Analyze the given JSON documents carefully.
2. Identify and extract only the information relevant to answer the user's question.
3. Summarize relevant tables, columns, keys, and descriptions into a clear and concise text.
4. Exclude any irrelevant or low-relevance details.
5. Ensure factual accuracy; do not fabricate information.
6. Do NOT generate or suggest any SQL queries.
7. When it is possible, match the language of the query fields to the user's query. For example, if user's query is in spanish, use `spanish_...`.
8. Match the language of the summary to the user's query.

Input:
- Retrieval documents (JSON): {retrieval_documents}

Output:
- A concise summary of relevant context—focused on tables, columns, keys, and descriptions—that will help generate an accurate SQL query. Don't suggest any query, only bring information in plain text. 
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{user_query}")
])

summarizer_chain = prompt | llm | StrOutputParser()

contexto_resumido = summarizer_chain.invoke({
    'user_query': QUERY,
    'retrieval_documents': json.dumps(openai_relevant_docs, indent=2, ensure_ascii=False)
})

Markdown(contexto_resumido)

Para obtener información sobre los 10 artículos más comprados, se pueden considerar las siguientes tablas y columnas relevantes de la base de datos `adventure_works_dw` en el esquema `sales`:

1. **Tabla: fact_sales**
   - **Descripción**: Contiene el detalle de las órdenes de ventas entregadas, con granularidad a nivel de línea.
   - **Claves Primarias**: 
     - sales_order_number
     - sales_order_line_number
   - **Columnas relevantes**:
     - **order_quantity**: Cantidad vendida de cada producto.
     - **product_key**: Clave del producto (clave foránea que se relaciona con la tabla `dim_product`).

2. **Tabla: dim_product**
   - **Descripción**: Almacena la información detallada de cada producto vendido.
   - **Clave Primaria**: 
     - product_key
   - **Columnas relevantes**:
     - **english_product_name**: Nombre del producto en inglés.
     - **spanish_product_name**: Nombre del producto en español.
     - **list_price**: Precio de venta del producto en USD.

Para determinar los artículos más comprados, se puede sumar la columna `order_quantity` de la tabla `fact_sales` agrupada por `product_key`, y luego unirse con la tabla `dim_product` para obtener los nombres de los productos.

#### LangChain: ChromaDB + Google GenAI Embeddings

Ahora haremos una segunda implementación de prueba de Chroma valiéndonos de funcionalidades de la librería LangChain, usando un modelo de embeddings de Google Gemini a través de Gemini API, pero trabajando con VertexAI de fondo, lo que nos permite asegurar que los datos permanezcan en la region / location deseada.

Dado que la clase implementada por LangChain para utilizar embeddings de Gemini API no soporta usar VertexAI por detrás, hemos procedido a implementar una clase personalizada que sí nos los permite, a la que hemos llamado `GenAIExtendedEmbeddingFunction`.

Generamos una instancia de la clase y verificamos que los parámetros son los esperados:

In [12]:
genai_embeddings = GenAIExtendedEmbeddingFunction(model= GENAI_EMBEDDING_MODEL)
print(f'VertexAI: {genai_embeddings.genai_client._api_client.vertexai}')
print(f'Project: {genai_embeddings.genai_client._api_client.project}')
print(f'Location: {genai_embeddings.genai_client._api_client.location}')
print(f'Auth Tokens: {genai_embeddings.genai_client.auth_tokens}')

VertexAI: True
Project: gen-lang-client-0151767776
Location: europe-west1
Auth Tokens: <google.genai.tokens.Tokens object at 0x0000017DEA26A710>


El próximo paso es crear un cliente persistente de ChromaDB, e inicializar las colecciones tanto a nivel de tablas como de columnas. Al inicializar las colecciones, especificaremos que queremos usar como métrica de distancia la distancia de Coseno, que se calcula de la siguiente manera:

$$Distancia = 1 - \frac{A \cdot B}{\|A\| \|B\|}$$

Donde:
* $A \cdot B$ es el producto punto (producto escalar) de los vectores $A$ y $B$.
* $∥A∥$ y $∥B∥$ son las magnitudes (o normas euclidianas) de los vectores $A$ y $B$, respectivamente.
​

La métrica **Distancia del Coseno** mide solo el ángulo entre vectores, lo que lo hace ideal para embeddings de texto, y particularmente para modelos de embeddings como el que usaremos [(`gemini-embedding-001`) cuyos valores vienen normalizados](https://ai.google.dev/gemini-api/docs/embeddings#quality-for-smaller-dimensions).

Como es fácil ver en su expresión, esta métrica de distancia devuelve valores en el rango $[0, 2]$. Cuanto más cerca de 0, más similares serán los vectores $A$ y $B$. Esto nos deja en la necesidad de definir una métrica de `relevance_score` personalizada, que nos permita reescalar los resultados en valores comprendidos entre $[0, 1]$, siendo los valores más cercanos a 1 indicativo de mayor similitud entre los vectores. Esta métrica la hemos definido en la función `cosine_distance_relevance_score_fn` de la siguiente manera:

$$R_{score} = 1 - \frac{Distancia}{2}$$

In [13]:
genai_embeddings = GenAIExtendedEmbeddingFunction(GENAI_EMBEDDING_MODEL)

mdl_hierarchical_genai_collection = MdlHierarchicalChromaCollections(
    collection_names= (TABLES_SUMMARY_COLLECTION_NAME, COLUMNS_COLLECTION_NAME),
    create_collections_if_not_exists= True,
    clear_collections= True,
    embedding_function= genai_embeddings,
    collections_configuration= {'hnsw': {'space': 'cosine'}}
)

mdl_hierarchical_genai_collection.add_documents(documents= (table_chunks, column_chunks))

(['427a1063-be9c-4cbf-a799-6bbc299e311f',
  'ec141c68-7f70-4798-b535-89f210517b27',
  '5f915455-cb64-45ff-8662-59b92e5b79a7',
  '3ded2fe5-7a89-4e82-86ca-9f677c13bd80',
  'df28f29d-594b-485e-91e8-84f0ed2405e4',
  '36845231-b072-4bcb-9f19-ef298119e2e2',
  'cc03e2fd-d7c4-42c5-bd6c-aa05977ff1e5',
  '61d35dc6-5611-43c7-980f-ce1d3ca443be',
  'f1053ef3-cef7-4ed9-83f6-912b4f022b1d',
  '6653bb58-4980-405e-9054-4e454af25fdf'],
 ['423bb90a-12b8-494a-8820-5650bfb2214b',
  'c28acb2d-1cdb-4a87-ac0f-1a00f0495c18',
  'cabbb2d5-aba6-40e4-906b-8c9fc54e7b77',
  '33df9c66-f890-48de-94d0-c93cb59fce52',
  'cc8e42d6-adcc-4ef2-9d46-16c625cb9c7f',
  '31e42ce5-4c35-410b-ade7-3940ab34efd7',
  '5ff31561-ac7c-4a5c-9336-a2d1e40a86a7',
  '88a18721-4b33-40a9-89df-a576eca8e921',
  '90127f9b-ace2-4a10-8be7-71ee1913edcb',
  'bf98eb49-e76f-4991-8e4c-ef5f265ae568',
  '60950c74-579f-4b6c-a2b5-1955edc68e2b',
  'f86ae0a9-9368-48a7-b40b-d7c47d21bcdb',
  'a586252e-8075-451e-b2b2-2e064815402b',
  '1373b85b-3712-4adf-b09b-0caf4a

##### Hierarchical RAG

Dada la naturaleza de nuestras colecciones, donde las columnas existen dentro del contexto de cada tabla, nos proponemos a utilizar la técnica de [**RAG jerárquico**](https://github.com/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/hierarchical_indices.ipynb) sobre las coleciones creadas. Esto significa que, primero buscaremos las tablas que sean más relevantes semánticamente para la query del usuario (en la coleción `tables_summary`) y, luego, para cada tabla obtenida, buscaremos sus columnas más relevantes (en la coleción `collumns`, filtrando la metada para que coincida con el dato de la tabla):

In [17]:
QUERY = 'Los 10 artículos más comprados.'

genai_relevant_docs = mdl_hierarchical_genai_collection.hierarchical_similarity_search(
    queries= QUERY,
    k_tables= 10,
    tables_score_threshold= 0.75,
    k_columns= 15,
    columns_score_threshold= 0.75
)

print(json.dumps(genai_relevant_docs, indent=2, ensure_ascii=False))

No relevant docs were retrieved using the relevance score threshold 0.75


[
  [
    {
      "table_summary": {
        "content": "Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promot

In [18]:
print(genai_relevant_docs)

[[{'table_summary': {'content': 'Database: adventure_works_dw\nSchema: sales\nTable: fact_sales\nTable description: Tabla de hechos que contiene el detalle de las ordenes de ventas que ya han sido entregadas, con una granularidad a nivel de línea, mostrando siempre la última versión de cada pedido. SIEMPRE que se soliciten datos de ventas, como cantidades vendidas, importe de ventas, costes de ventas, impuestos, costes de envío, deberán ser obtenidos de esta tabla. Permite hacer desgloses a nivel de productos, clientes, tiendas/distribuidores, división territorial, promociones aplicadas y vendedores involucrados.\nTable PRIMARY KEY:\n- sales_order_number\n- sales_order_line_number\nTable FOREIGN KEYS (Column name, Reference):\n- (product_key, sales.dim_product.product_key)\n- (reseller_key, sales.dim_reseller.reseller_key)\n- (employee_key, sales.dim_sales_person.employee_key)\n- (customer_key, sales.dim_customer.customer_key)\n- (promotion_key, sales.dim_promotion.promotion_key)\n- (s

In [19]:
llm = AzureChatOpenAI(model='gpt-4o-mini', temperature=0)

SYSTEM_PROMPT = """
You are a context summarizer that receives retrieval documents in JSON format.
Your role is to:

1. Analyze the given JSON documents carefully.
2. Identify and extract only the information relevant to answer the user's question.
3. Summarize relevant tables, columns, keys, and descriptions into a clear and concise text.
4. Exclude any irrelevant or low-relevance details.
5. Ensure factual accuracy; do not fabricate information.
6. Do NOT generate or suggest any SQL queries.
7. When it is possible, match the language of the query fields to the user's query. For example, if user's query is in spanish, use `spanish_...`.
8. Match the language of the summary to the user's query.

Input:
- Retrieval documents (JSON): {retrieval_documents}

Output:
- A concise summary of relevant context—focused on tables, columns, keys, and descriptions—that will help generate an accurate SQL query. Don't suggest any query, only bring information in plain text. 
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{user_query}")
])

summarizer_chain = prompt | llm | StrOutputParser()

contexto_resumido = summarizer_chain.invoke({
    'user_query': QUERY,
    'retrieval_documents': json.dumps(genai_relevant_docs, indent=2, ensure_ascii=False)
})

Markdown(contexto_resumido)

Para obtener información sobre los 10 artículos más comprados, se pueden considerar las siguientes tablas y columnas relevantes de la base de datos `adventure_works_dw`:

1. **Tabla: fact_sales**
   - **Descripción**: Contiene el detalle de las órdenes de ventas entregadas, con información a nivel de línea.
   - **Claves Primarias**: 
     - sales_order_number
     - sales_order_line_number
   - **Columnas relevantes**:
     - **order_quantity**: Cantidad vendida de cada producto.
     - **product_key**: Clave del producto (relacionada con la tabla `dim_product`).

2. **Tabla: dim_product**
   - **Descripción**: Almacena información detallada de cada producto vendido.
   - **Clave Primaria**: 
     - product_key
   - **Columnas relevantes**:
     - **english_product_name**: Nombre del producto en inglés.
     - **spanish_product_name**: Nombre del producto en español.
     - **list_price**: Precio de venta del producto en USD.

Para determinar los artículos más comprados, se puede utilizar la columna `order_quantity` de la tabla `fact_sales` para sumar las cantidades vendidas y relacionar con la tabla `dim_product` para obtener los nombres de los productos.

#### Conclusiones e Implementación Final

Como podemos ver, siguiendo el mismo proceso para ambos modelos de embeddings, el modelo `gemini-embedding-001` de Google, aunque parece devolver resultados similares que los que se obtienen con `text-embedding-3-large` de OpenAI, consigue mejores resultados en términos de relevancia. Esto nos hace pensar que es mejor al capturar la semántica de nuestro documentos frente a la de las consultas.

Esto puede deberse, en gran medida, a que , aunque ambos modelos presentan un **tamaño de embeddings de 3072**, el modelo de Google presenta un rendimiento destacable en tareas multilingües, como puede apreciarse en el [paper de su presentación](https://arxiv.org/html/2503.07891v1). Este es un punto no menor al trabajar nuestra documentación, ya que la misma mezcla conceptos en inglés (nombres de tablas y campos) y en español (descripciones), como asi también al momento de consultar, que podrá hacerse en cualquiera de estos 2 idiomas de manera indistinta.

Teniendo en cuenta estas conclusiones, finalmente, **procedemos a implementar nuestras colecciones de `ChromaDB` de manera definitiva usando el modelo `gemini-embedding-001`**.

In [ ]:
genai_embeddings = GenAIExtendedEmbeddingFunction(GENAI_EMBEDDING_MODEL)

mdl_hierarchical_genai_collection = MdlHierarchicalChromaCollections(
    persist_directory= CHROMA_DB_PATH,
    collection_names= (TABLES_SUMMARY_COLLECTION_NAME, COLUMNS_COLLECTION_NAME),
    create_collections_if_not_exists= True,
    embedding_function= genai_embeddings,
    collections_configuration= {'hnsw': {'space': 'cosine'}}
)

mdl_hierarchical_genai_collection.add_documents(documents= (table_chunks, column_chunks))

(['427a1063-be9c-4cbf-a799-6bbc299e311f',
  'ec141c68-7f70-4798-b535-89f210517b27',
  '5f915455-cb64-45ff-8662-59b92e5b79a7',
  '3ded2fe5-7a89-4e82-86ca-9f677c13bd80',
  'df28f29d-594b-485e-91e8-84f0ed2405e4',
  '36845231-b072-4bcb-9f19-ef298119e2e2',
  'cc03e2fd-d7c4-42c5-bd6c-aa05977ff1e5',
  '61d35dc6-5611-43c7-980f-ce1d3ca443be',
  'f1053ef3-cef7-4ed9-83f6-912b4f022b1d',
  '6653bb58-4980-405e-9054-4e454af25fdf'],
 ['423bb90a-12b8-494a-8820-5650bfb2214b',
  'c28acb2d-1cdb-4a87-ac0f-1a00f0495c18',
  'cabbb2d5-aba6-40e4-906b-8c9fc54e7b77',
  '33df9c66-f890-48de-94d0-c93cb59fce52',
  'cc8e42d6-adcc-4ef2-9d46-16c625cb9c7f',
  '31e42ce5-4c35-410b-ade7-3940ab34efd7',
  '5ff31561-ac7c-4a5c-9336-a2d1e40a86a7',
  '88a18721-4b33-40a9-89df-a576eca8e921',
  '90127f9b-ace2-4a10-8be7-71ee1913edcb',
  'bf98eb49-e76f-4991-8e4c-ef5f265ae568',
  '60950c74-579f-4b6c-a2b5-1955edc68e2b',
  'f86ae0a9-9368-48a7-b40b-d7c47d21bcdb',
  'a586252e-8075-451e-b2b2-2e064815402b',
  '1373b85b-3712-4adf-b09b-0caf4a

---

## Fichero MarkDown con Reglas de Negocio

El objetivo de esta sección es procesar un fichero MarkDown que contien lógica de negocio. Para esto, debemos cargarlo, generar los chunks de una manera que sean semánticamente relevantes, generar su metadata si creemos que corresponde, y, finalmente generar sus embeddings y almacenarlos en una colescción de nuestra base de datos vectorial ChromaDB.

Como primer paso, procedemos a cargar el fichero y a ver un poco de su aspecto:

### Chunking

Ahora definimos algunas funciones que nos permitirán actualizar el contenido MarkDown de nuestro fichero, y de sus chunks. Esto es necesario debido a que la clase nativa de LangChain `MarkdownHeaderTextSplitter` no solo que no "limpia" el contenido del MarkDown, si no que también internamente durante su procesamiento utiliza `.strip()` sobre las líneas que conforman el texto MarkDown, lo que hace que se pierda la jerarquía del mismo.

Utilizaremos como primer método de chunking un `MarkdownHeaderTextSplitter`, para luego aplicar un `RecursiveCharacterTextSplitter` que nos permita llevar los chunks al tamaño deseado. Finalmente, añadiremos en la metadata de cada chunk un `id` que lo identifique de forma única dentro de la combinación de headers que presente en la metadata.

In [18]:
md_splitter = ExtendedMarkdownSplitter(
    chunk_size= BUSINESS_RULES_CHUNK_SIZE,
    chunk_overlap= BUSINESS_RULES_CHUNK_OVERLAP
)

md_text = md_splitter._load_md_text(BUSINESS_RULES_PATH, encoding='utf-8')
Markdown(md_text[500:2000])

 y la operacionalización de la inteligencia de negocio.

## 1. Representación de Entidades y Preferencias Lingüísticas

Cuando se solicita conceptualmente una entidad, como "productos", "clientes", "promociones" o "revendedores", se establece como principio operativo priorizar los campos que finalizan en `_name` para su adecuada representación en las salidas de las consultas SQL. Adicionalmente, se procurará mantener la coherencia con el idioma en que la consulta original fue formulada.

- **Principio Operativo:** Para la visualización de entidades en los resultados de consultas, se debe seleccionar el campo `_name` que mejor represente la entidad. En el caso de campos bilingües donde la versión en español (`spanish_..._name`, `spanish_..._description`) pueda ser nula y la consulta se realice en español, se utilizará la versión en inglés como alternativa (`fallback`) para asegurar la disponibilidad del dato y mantener la representación de la entidad.
- **Lógica SQL (Ejemplo para `dim_product`):**

    ```sql
    -- Selección del nombre del producto en español o, si es NULL, en inglés, para su representación
    SELECT
        product_key,
        COALESCE(spanish_product_name, english_product_name) AS producto_nombre,
        COALESCE(spanish_product_category_name, english_product_category_name) AS producto_category,
        COALESCE(spanish_product_subcategory_name, english_product_subcategory_name) AS producto_subcategoria
    FROM
        sales.dim_product;
    ```

    Es

Veamos ahora cuántos chunks genera nuestro splitter, y el aspecto de estos:

In [19]:
business_logic_chunks = md_splitter.split_documents(BUSINESS_RULES_PATH, encoding='utf-8')

print(f'{len(business_logic_chunks)=}')
print('-'*100)
print(business_logic_chunks)

len(business_logic_chunks)=24
----------------------------------------------------------------------------------------------------
[Document(id='75a462d9-a1ac-43fb-ba1f-8c848077b6fd', metadata={'2_subtitle': '1. Representación de Entidades y Preferencias Lingüísticas', 'file_name': 'business_rules.md', '3_section': 'N/A', 'headers_chunk_id': 0, 'headers_comb_total': 2, 'chunk_overlap': 300}, page_content='1. Representación de Entidades y Preferencias Lingüísticas\n\nCuando se solicita conceptualmente una entidad, como "productos", "clientes", "promociones" o "revendedores", se establece como principio operativo priorizar los campos que finalizan en `_name` para su adecuada representación en las salidas de las consultas SQL. Adicionalmente, se procurará mantener la coherencia con el idioma en que la consulta original fue formulada.\n\n- Principio Operativo: Para la visualización de entidades en los resultados de consultas, se debe seleccionar el campo `_name` que mejor represente la ent

### LangChain: ChromaDB + Google GenAI Embeddings

Ahora sí, procedemos a generar los embeddings de cada chunk y a almacenarlos en nuestra base de datos vectorial de `ChromaDB`. Esta vez, siguiendo la línea de las conclusiones obtenidas en el apartado anterior, utilizaremos un modelo de embeddings de Google Gemini a través de Gemini API, pero trabajando con VertexAI de fondo, lo que nos permite asegurar que los datos permanezcan en la region / location deseada.

Dado que la clase implementada por LangChain para utilizar embeddings de Gemini API no soporta usar VertexAI por detrás, nuevamente, utilizaremos la clase `GenAIExtendedEmbeddingFunction` creada anteriormente.

Generamos una instancia de la clase y verificamos que los parámetros son los esperados:

In [20]:
genai_embeddings = GenAIExtendedEmbeddingFunction(model= GENAI_EMBEDDING_MODEL)
print(f'VertexAI: {genai_embeddings.genai_client._api_client.vertexai}')
print(f'Project: {genai_embeddings.genai_client._api_client.project}')
print(f'Location: {genai_embeddings.genai_client._api_client.location}')
print(f'Auth Tokens: {genai_embeddings.genai_client.auth_tokens}')

VertexAI: True
Project: gen-lang-client-0151767776
Location: europe-west1
Auth Tokens: <google.genai.tokens.Tokens object at 0x0000023A9064E390>


El próximo paso es crear un cliente persistente de ChromaDB, e inicializar la colección correspondiente con la lógica de negocio. Al inicializar la colección, nuevamente, especificaremos que queremos usar como métrica de distancia la distancia de Coseno.

In [ ]:
genai_embeddings = GenAIExtendedEmbeddingFunction(GENAI_EMBEDDING_MODEL)

business_logic_collection = ContextEnricherChromaCollection(
    persist_directory= CHROMA_DB_PATH,
    collection_name= BUSINESS_LOGIC_COLLECTION_NAME,
    create_collection_if_not_exists= True,
    embedding_function= genai_embeddings,
    collection_configuration= {'hnsw': {'space': 'cosine'}}
)

business_logic_collection.add_documents(documents= business_logic_chunks)

['75a462d9-a1ac-43fb-ba1f-8c848077b6fd',
 'fd623479-33b2-4908-9fd6-323968ae0268',
 'b6b81056-8610-48a2-a3a0-0b17a105e5e3',
 '6d8cadb4-11ca-49c4-a0fd-b458af90cab2',
 '625d5069-97bc-4bc9-89af-cc62eea5778c',
 'd0aafb3e-8f80-4d41-b3cb-5c850935e8c7',
 '094ddbf6-caa0-4bdf-9782-6e6476a64074',
 '6439c1dc-8dcd-4dd6-a4ee-3e72f2c4a071',
 '3cc5c632-d34c-41d6-bb76-28c321c247a4',
 'aa59d3e2-e477-4e48-b277-967c2c608d65',
 '59e63199-43b2-4f3e-ad33-92f054131f6e',
 '640a7784-ea71-4198-b117-baf8bc7b1941',
 'c65290cd-0676-45ef-8fba-060baf8a54c2',
 '79358930-da9e-42c8-a47d-aa2c96ac766f',
 'f8cd7f72-7944-474f-8703-5f8848dedf42',
 '84cd4872-e782-43ab-a11d-45eaa67cf125',
 '25ac0626-f0bf-49ed-a5cf-6ae651e1a82d',
 'e9f2d6f6-fbe7-40d6-a423-ed05d95a4863',
 '81ec14cb-2a53-4777-b9b3-aae50d03dbc1',
 '0b81ca82-a5cb-4e97-aebd-00a7f8c3a03d',
 'fa28a269-f5d3-4b01-bf3a-7d5f248a154b',
 '7248eb44-b763-484e-9fdb-74cb057d6877',
 'bc9ad748-6077-49e7-856d-e7b449f41eb2',
 'd444d1d7-45b3-4abe-9d8b-2c3a5d35847e']

Ahora lanzaremos una consulta a la colección creada, para ver que los resultados obtenidos muestren cierto nivel de relevancia. Para esto, como ya hemos hecho en el apartado anterior, utilizaremoscomo métrica personalizada de `relevance_score` la función `cosine_distance_relevance_score_fn`.

In [22]:
QUERY = 'Los 10 artículos más comprados.'

In [23]:
business_logic_score_results = business_logic_collection.search(
    queries= QUERY,
    search_type= 'similarity_score_threshold',
    k= 5,
    score_threshold= 0.8
)

business_logic_score_results

[(Document(id='aa59d3e2-e477-4e48-b277-967c2c608d65', metadata={'2_subtitle': '3. Métricas Clásicas del Retail y su Interpretación', 'headers_chunk_id': 1, 'headers_comb_total': 3, 'file_name': 'business_rules.md', '3_section': 'N/A', 'chunk_overlap': 300}, page_content='- Cuando se habla de productos "más vendidos", sin especificar más, generalmente nos referimos a la cantidad de unidades (`order_quantity`) vendidas. También puede ser relevante medir esto por el importe facturado (`sales_amount`) para identificar productos que, aunque se vendan en menor volumen, generan más ingresos. Pero, en este último caso se especificarán términos como "de mayor facturación".\n- Cuando se refiere a clientes, tiendas/distribuidores o vendedores que "más compran/venden", esto se mide por el importe facturado (`sales_amount`) o `net sales`. Adicionalmente, se puede considerar la frecuencia de compra (para clientes y distribuidores) o el número de pedidos gestionados (para vendedores) como métricas co

In [24]:
business_logic_score_results[1][0]

Document(id='3cc5c632-d34c-41d6-bb76-28c321c247a4', metadata={'file_name': 'business_rules.md', 'headers_comb_total': 3, '2_subtitle': '3. Métricas Clásicas del Retail y su Interpretación', '3_section': 'N/A', 'headers_chunk_id': 0, 'chunk_overlap': 300}, page_content='3. Métricas Clásicas del Retail y su Interpretación\n\nEs crucial entender cómo se miden las diferentes métricas en el contexto de negocio.')

In [25]:
business_logic_score_results[0][0]

Document(id='aa59d3e2-e477-4e48-b277-967c2c608d65', metadata={'2_subtitle': '3. Métricas Clásicas del Retail y su Interpretación', 'headers_chunk_id': 1, 'headers_comb_total': 3, 'file_name': 'business_rules.md', '3_section': 'N/A', 'chunk_overlap': 300}, page_content='- Cuando se habla de productos "más vendidos", sin especificar más, generalmente nos referimos a la cantidad de unidades (`order_quantity`) vendidas. También puede ser relevante medir esto por el importe facturado (`sales_amount`) para identificar productos que, aunque se vendan en menor volumen, generan más ingresos. Pero, en este último caso se especificarán términos como "de mayor facturación".\n- Cuando se refiere a clientes, tiendas/distribuidores o vendedores que "más compran/venden", esto se mide por el importe facturado (`sales_amount`) o `net sales`. Adicionalmente, se puede considerar la frecuencia de compra (para clientes y distribuidores) o el número de pedidos gestionados (para vendedores) como métricas comp

#### Maximal Marginal Relevance (MMR)

Como podemos ver, los resultados obtenidos muestran valores altos de relevancia, por lo que a priori parece funcionar correctamente. Dado que para este caso, a diferencia del anterior, estamos trabajando con chunks que presentan overlapping, corremos el riesgo de que nuestros resultados sean redundantes, como se muestra en las 2 celdas anteriores. Ante esta situación, parece una buena idea utilizar `max_marginal_relevance_search`. 

Esta búsqueda aplica la técnica de [***Maximal Marginal Relevance (MMR)***](https://aclanthology.org/X98-1025/), mediante la que se intenta reducir la redundancia en los resultados mientras, al mismo tiempo, se mantiene la relevancia de los mismos. Es decir, un documento presentará una alta Relevancia Marginal si es a la vez relevante para la query y presenta una similitud reducida con los documentos previamente seleccionados.

In [26]:
business_logic_mmr_results = business_logic_collection.search(
    queries= QUERY,
    search_type= 'mmr',
    k= 5,
    fetch_k= 25,
    lambda_mult= 0.5
)

business_logic_mmr_results

[Document(id='aa59d3e2-e477-4e48-b277-967c2c608d65', metadata={'headers_comb_total': 3, '3_section': 'N/A', 'headers_chunk_id': 1, 'chunk_overlap': 300, '2_subtitle': '3. Métricas Clásicas del Retail y su Interpretación', 'file_name': 'business_rules.md'}, page_content='- Cuando se habla de productos "más vendidos", sin especificar más, generalmente nos referimos a la cantidad de unidades (`order_quantity`) vendidas. También puede ser relevante medir esto por el importe facturado (`sales_amount`) para identificar productos que, aunque se vendan en menor volumen, generan más ingresos. Pero, en este último caso se especificarán términos como "de mayor facturación".\n- Cuando se refiere a clientes, tiendas/distribuidores o vendedores que "más compran/venden", esto se mide por el importe facturado (`sales_amount`) o `net sales`. Adicionalmente, se puede considerar la frecuencia de compra (para clientes y distribuidores) o el número de pedidos gestionados (para vendedores) como métricas com

#### Context Enrichment Window

Como podemos observar en el resultado anterior, Parece ser que los 5 chunks que hemos obtenido, a diferencia de lo que ocurría antes, ninguno pertenece a la misma jerarquía (mismo `2_subtitle` y `3_section`), por lo que parece estar funcionando mejor. Utilizaremos entonces esta técnica, y la combinaremos con una técnica de [**RAG de contexto eriquecido**](https://github.com/NirDiamant/RAG_Techniques/blob/main/all_rag_techniques/context_enrichment_window_around_chunk.ipynb), lo que nos permitirá ampliar el contexto que circunda a los chunks obtenidos.

Para lograr esto, hemos implementado una clase personalizada que contine toda la lógica necesaria para generar un contexto enriquecido. Está definida en la sección `Clases personalizadas`, bajo el nombre `DocumentContextEnricher`.

In [27]:
business_logic_enriched_context_results = business_logic_collection.enriched_context_search(
    queries= QUERY,
    merge_results= True,
    context_window_size= 3,
    search_type= 'mmr',
    k= 10,
    fetch_k= 25,
    lambda_mult= 0.5
)

print(business_logic_enriched_context_results[0])

3. Métricas Clásicas del Retail y su Interpretación

Es crucial entender cómo se miden las diferentes métricas en el contexto de negocio.- Cuando se habla de productos "más vendidos", sin especificar más, generalmente nos referimos a la cantidad de unidades (`order_quantity`) vendidas. También puede ser relevante medir esto por el importe facturado (`sales_amount`) para identificar productos que, aunque se vendan en menor volumen, generan más ingresos. Pero, en este último caso se especificarán términos como "de mayor facturación".
- Cuando se refiere a clientes, tiendas/distribuidores o vendedores que "más compran/venden", esto se mide por el importe facturado (`sales_amount`) o `net sales`. Adicionalmente, se puede considerar la frecuencia de compra (para clientes y distribuidores) o el número de pedidos gestionados (para vendedores) como métricas complementarias.
- Para el análisis de rentabilidad, la métrica clave es el margen bruto (`gross margin`) o el porcentaje de margen bruto 

In [28]:
print(business_logic_enriched_context_results)

['3. Métricas Clásicas del Retail y su Interpretación\n\nEs crucial entender cómo se miden las diferentes métricas en el contexto de negocio.- Cuando se habla de productos "más vendidos", sin especificar más, generalmente nos referimos a la cantidad de unidades (`order_quantity`) vendidas. También puede ser relevante medir esto por el importe facturado (`sales_amount`) para identificar productos que, aunque se vendan en menor volumen, generan más ingresos. Pero, en este último caso se especificarán términos como "de mayor facturación".\n- Cuando se refiere a clientes, tiendas/distribuidores o vendedores que "más compran/venden", esto se mide por el importe facturado (`sales_amount`) o `net sales`. Adicionalmente, se puede considerar la frecuencia de compra (para clientes y distribuidores) o el número de pedidos gestionados (para vendedores) como métricas complementarias.\n- Para el análisis de rentabilidad, la métrica clave es el margen bruto (`gross margin`) o el porcentaje de margen 

In [29]:
llm = AzureChatOpenAI(model='gpt-4o-mini', temperature=0.25)

SYSTEM_PROMPT = """
You are a context summarizer that receives retrieval documents.
Your role is to:

1. Analyze the given retrieval documents, which contain business logic information.
2. Identify and extract only the business logic rules and definitions relevant to the user's question. This includes metrics, calculations, entity definitions, and specific operational principles.
3. Summarize these business logics into a clear, concise, and easy-to-understand text.
4. Exclude any irrelevant details like table or column names, keys, or direct SQL query examples.
5. Ensure factual accuracy; do not fabricate information.
6. The summary should be in plain text, exclusively focused on explaining the business rules. Do not mention any technical or database-related details.
7. Match the language of the summary to the user's query.

Input:
- Retrieval documents: {retrieval_documents}

Output:
- A concise summary of the business logic relevant to the user's query. The summary should be focused on explaining the business rules and definitions in a non-technical manner.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{user_query}")
])

summarizer_chain = prompt | llm | StrOutputParser()

contexto_resumido = summarizer_chain.invoke({
    'user_query': QUERY,
    'retrieval_documents': business_logic_enriched_context_results
})

Markdown(contexto_resumido)

Para identificar los 10 artículos más comprados, se considera la cantidad de unidades vendidas como la métrica principal. Alternativamente, también se puede evaluar el importe facturado de cada producto para determinar cuáles generan más ingresos, aunque en este caso se utilizaría el término "de mayor facturación". Esta evaluación permite a las empresas entender qué productos tienen mayor demanda y contribuyen significativamente a sus ingresos.

---

## Fichero JSON con Ejemplos de Consultas

El objetivo de esta sección es procesar un fichero JSON que contien ejemplos variados, en los que se indica la pregunta del usuario, bien en español o bien en inglés, y una query que respondería a la consulta del usuario.

Para esto, nos valdremos de las clases personalizadas que hemos creado. Utilizaremos `JsonExamplesSplitter` para cargar los datos del fichero y generar los chunks con su correspondiente metadata, y `ExtendedChromaCollection` para gestionar la colección correspondiente de Chroma DB donde almacenaremos y consultaremos estos chunks.

### Chunking

En este caso, a diferencia de los anteriores, los "documentos" de los que debemos generar los embeddings son las consultas del usuario, mientras que en la metadata almacenaremos la query que responde a dicha consulta, como también el idioma original de la consulta, por si deseamos filtrar.

In [3]:
examples_splitter = JsonExamplesSplitter(page_content_key= 'user_query')

examples_chunks = examples_splitter.split_documents(EXAMPLES_PATH, encoding= 'utf-8')

print(f'{len(examples_chunks)=}')
print('-'*100)
print(examples_chunks)

len(examples_chunks)=21
----------------------------------------------------------------------------------------------------
[Document(id='d8fce838-4457-4b9b-8984-21ab3d74e756', metadata={'file_name': 'query_examples.json', 'sql_query': "SELECT\n  COALESCE(spanish_product_name, english_product_name) AS producto,\n  list_price AS precio_de_lista,\n  standard_cost AS coste_estandar,\n  spanish_product_category_name AS categoria,\n  spanish_product_subcategory_name AS subcategoria,\n  CASE\n    WHEN list_price >= 1000 THEN 'Premium'\n    WHEN list_price >= 200 AND list_price < 1000 THEN 'Estándar'\n    ELSE 'Económico'\n  END AS escala_precios\nFROM\n  sales.dim_product\nORDER BY\n  product_category,\n  product_subcategory;", 'language': 'ES'}, page_content='Necesito saber para cada producto su nombre, su precio de lista, su coste, su categoría y subcategoría, y escala de precios al que pertenece, ordenado por categoría y subcategoría.'), Document(id='67ff5cbf-f93a-4436-8434-f24525ce30d8'

Veamos un ejemplo en español y uno en inglés:

In [4]:
first_spanish_chunk = next((chunk for chunk in examples_chunks if chunk.metadata.get('language') == 'ES'), None)

title = 'id:'
print(title)
print('-' * (len(title)+1))
print(first_spanish_chunk.id, end= '\n'*2)

title = 'page_content:'
print(title)
print('-' * (len(title)+1))
print(first_spanish_chunk.page_content, end= '\n'*2)

title = 'query:'
print(title)
print('-' * (len(title)+1))
print(first_spanish_chunk.metadata.get('sql_query'))

id:
----
d8fce838-4457-4b9b-8984-21ab3d74e756

page_content:
--------------
Necesito saber para cada producto su nombre, su precio de lista, su coste, su categoría y subcategoría, y escala de precios al que pertenece, ordenado por categoría y subcategoría.

query:
-------
SELECT
  COALESCE(spanish_product_name, english_product_name) AS producto,
  list_price AS precio_de_lista,
  standard_cost AS coste_estandar,
  spanish_product_category_name AS categoria,
  spanish_product_subcategory_name AS subcategoria,
  CASE
    WHEN list_price >= 1000 THEN 'Premium'
    WHEN list_price >= 200 AND list_price < 1000 THEN 'Estándar'
    ELSE 'Económico'
  END AS escala_precios
FROM
  sales.dim_product
ORDER BY
  product_category,
  product_subcategory;


In [5]:
first_english_chunk = next((chunk for chunk in examples_chunks if chunk.metadata.get('language') == 'EN'), None)

title = 'id:'
print(title)
print('-' * (len(title)+1))
print(first_english_chunk.id, end= '\n'*2)

title = 'page_content:'
print(title)
print('-' * (len(title)+1))
print(first_english_chunk.page_content, end= '\n'*2)

title = 'query:'
print(title)
print('-' * (len(title)+1))
print(first_english_chunk.metadata.get('sql_query'))

id:
----
618499d8-a97c-47b3-b5db-882d6223f07d

page_content:
--------------
Show me the average shipping time, delivery time, and total order processing time in days for the last quarter.

query:
-------
SELECT
  AVG(DATE_PART('day', ship_date::TIMESTAMP - order_date::TIMESTAMP)) AS avg_shipping_time_days,
  AVG(DATE_PART('day', due_date::TIMESTAMP - ship_date::TIMESTAMP)) AS avg_delivery_time_days,
  AVG(DATE_PART('day', due_date::TIMESTAMP - order_date::TIMESTAMP)) AS avg_total_order_processing_time_days
FROM
  sales.fact_sales
WHERE
  order_date BETWEEN DATE_TRUNC('quarter', CURRENT_DATE - INTERVAL '3 months') AND DATE_TRUNC('quarter', CURRENT_DATE) - INTERVAL '1 day';


### LangChain: ChromaDB + Google GenAI Embeddings

Ahora sí, procedemos a generar los embeddings de cada chunk y a almacenarlos en nuestra base de datos vectorial de `ChromaDB`. Siguiendo la línea de apartados anteriores, utilizaremos nuevamente el modelo de embeddings de Google Gemini a través de Gemini API, siempre trabajando con VertexAI de fondo, lo que nos permite asegurar que los datos permanezcan en la region / location deseada.

Dado que la clase implementada por LangChain para utilizar embeddings de Gemini API no soporta usar VertexAI por detrás, nuevamente, utilizaremos la clase `GenAIExtendedEmbeddingFunction` creada anteriormente.

Generamos una instancia de la clase y verificamos que los parámetros son los esperados:

In [6]:
genai_embeddings = GenAIExtendedEmbeddingFunction(model= GENAI_EMBEDDING_MODEL)
print(f'VertexAI: {genai_embeddings.genai_client._api_client.vertexai}')
print(f'Project: {genai_embeddings.genai_client._api_client.project}')
print(f'Location: {genai_embeddings.genai_client._api_client.location}')
print(f'Auth Tokens: {genai_embeddings.genai_client.auth_tokens}')

VertexAI: True
Project: gen-lang-client-0151767776
Location: europe-west1
Auth Tokens: <google.genai.tokens.Tokens object at 0x0000017CE5498250>


El próximo paso es crear un cliente persistente de ChromaDB, e inicializar la colección correspondiente con ejemplos de queries. Al inicializar la colección, nuevamente, especificaremos que queremos usar como métrica de distancia la distancia de Coseno.

In [ ]:
genai_embeddings = GenAIExtendedEmbeddingFunction(GENAI_EMBEDDING_MODEL)

examples_collection = ExamplesChromaCollection(
    persist_directory= CHROMA_DB_PATH,
    collection_name= EXAMPLES_COLLECTION_NAME,
    create_collection_if_not_exists= True,
    embedding_function= genai_embeddings,
    collection_configuration= {'hnsw': {'space': 'cosine'}}
)

examples_collection.add_documents(documents= examples_chunks)

['d8fce838-4457-4b9b-8984-21ab3d74e756',
 '67ff5cbf-f93a-4436-8434-f24525ce30d8',
 '28e85081-2738-440d-aa64-424227971e4b',
 '991cac17-1d9e-402e-be10-9ab834dcd88f',
 'c3154e54-b358-4ff5-acad-ddf2839b5ec7',
 '45cc7247-46dd-44c2-8baa-aab5bda6bd88',
 '8be6862c-2c70-4d58-b7fd-8120c456f121',
 '136adf47-62dd-4a68-95cb-5f9a0fb53570',
 'e374f5e9-48d7-4e07-bbf4-5509749ccd55',
 '00e54ab4-fb49-4f1b-8c20-9662a100fadf',
 'd9deafaa-5842-4212-964a-00b624b66467',
 '618499d8-a97c-47b3-b5db-882d6223f07d',
 'ad617cff-6972-4b6d-997d-911a02886422',
 'ab0a6c3f-3e0e-4102-a16a-c651763b42dc',
 '1dbac876-218f-4db5-91e4-c15404151868',
 'a1f97722-b208-488a-b8fe-045c95681efc',
 '65ae6760-2ebd-4fb6-a760-a8f137f9acb2',
 '47a63d33-b4db-48ae-a0a6-d03aaddce256',
 '3b383017-7a31-4aed-b275-72e401fa07b5',
 'c3e941b7-c326-4d89-905e-42ee1b4041f4',
 'aa3d2d97-2339-4294-b5d8-eeb9076643c4']

Dado que actualmente tenemos una cantidad reducida de ejemplos en nuestra base de datos, parece adecuado utilizar una búsqueda por score de relevancia.

En caso de que luego se habilite la carga de una cantidad mayor de ejemplos, podríamos correr el riesgo de que tengamos muchas consultas similares. En ese caso, parecería una buena idea utilizar `max_marginal_relevance_search`.

Como ya comentamos anteriormente, esta búsqueda aplica la técnica de [***Maximal Marginal Relevance (MMR)***](https://aclanthology.org/X98-1025/), mediante la que se intenta reducir la redundancia en los resultados mientras, al mismo tiempo, se mantiene la relevancia de los mismos. Es decir, un documento presentará una alta Relevancia Marginal si es a la vez relevante para la query y presenta una similitud reducida con los documentos previamente seleccionados.

In [8]:
QUERY = 'Los 10 artículos más comprados.'

examples_results = examples_collection.search(
    queries= QUERY,
    search_type= 'similarity_score_threshold',
    k = 2
)

examples_results

[(Document(id='c3154e54-b358-4ff5-acad-ddf2839b5ec7', metadata={'language': 'ES', 'sql_query': 'SELECT\n  COALESCE(dp.spanish_product_name, dp.english_product_name) AS producto,\n  dp.spanish_product_category_name AS categoria,\n  dp.spanish_product_subcategory_name AS subcategoria,\n  SUM(fs.sales_amount) AS importe,\n  SUM(fs.order_quantity) AS cantidad,\n  SUM(fs.sales_amount) / NULLIF(SUM(fs.order_quantity), 0) AS precio_medio\nFROM\n  sales.fact_sales fs\n  JOIN sales.dim_product dp ON fs.product_key = dp.product_key\nWHERE\n  EXTRACT(YEAR FROM fs.order_date) = EXTRACT(YEAR FROM CURRENT_DATE) - 1\nGROUP BY\n  producto, categoria, subcategoria\nORDER BY\n  cantidad DESC\nLIMIT 10;', 'file_name': 'query_examples.json'}, page_content='Enumera los 10 productos más vendidos en el último año. Muestra el nombre del producto, su categoría y subcategoría, el importe de las ventas, la cantidad vendida, y el precio medio por unidad.'),
  0.8890863955020905),
 (Document(id='d9deafaa-5842-4212

In [9]:
first_result = examples_results[0][0]
score = examples_results[0][1]

title = 'id:'
print(title)
print('-' * (len(title)+1))
print(first_result.id, end= '\n'*2)

title = 'score:'
print(title)
print('-' * (len(title)+1))
print(score, end= '\n'*2)

title = 'page_content:'
print(title)
print('-' * (len(title)+1))
print(first_result.page_content, end= '\n'*2)

title = 'query:'
print(title)
print('-' * (len(title)+1))
print(first_result.metadata.get('sql_query'))

id:
----
c3154e54-b358-4ff5-acad-ddf2839b5ec7

score:
-------
0.8890863955020905

page_content:
--------------
Enumera los 10 productos más vendidos en el último año. Muestra el nombre del producto, su categoría y subcategoría, el importe de las ventas, la cantidad vendida, y el precio medio por unidad.

query:
-------
SELECT
  COALESCE(dp.spanish_product_name, dp.english_product_name) AS producto,
  dp.spanish_product_category_name AS categoria,
  dp.spanish_product_subcategory_name AS subcategoria,
  SUM(fs.sales_amount) AS importe,
  SUM(fs.order_quantity) AS cantidad,
  SUM(fs.sales_amount) / NULLIF(SUM(fs.order_quantity), 0) AS precio_medio
FROM
  sales.fact_sales fs
  JOIN sales.dim_product dp ON fs.product_key = dp.product_key
WHERE
  EXTRACT(YEAR FROM fs.order_date) = EXTRACT(YEAR FROM CURRENT_DATE) - 1
GROUP BY
  producto, categoria, subcategoria
ORDER BY
  cantidad DESC
LIMIT 10;


In [10]:
second_result = examples_results[1][0]
score = examples_results[1][1]

title = 'id:'
print(title)
print('-' * (len(title)+1))
print(second_result.id, end= '\n'*2)

title = 'score:'
print(title)
print('-' * (len(title)+1))
print(score, end= '\n'*2)

title = 'page_content:'
print(title)
print('-' * (len(title)+1))
print(second_result.page_content, end= '\n'*2)

title = 'query:'
print(title)
print('-' * (len(title)+1))
print(second_result.metadata.get('sql_query'))

id:
----
d9deafaa-5842-4212-964a-00b624b66467

score:
-------
0.8772459626197815

page_content:
--------------
Muéstrame los 10 productos más vendidos para el B2B en el último trimestre, excluyendo los Accesorios. Mes a mes indica cuántas unidades se vendieron, y muestra también a qué categoría y subcategoría pertenecen.

query:
-------
WITH 
  last_quarter AS (
    SELECT
      fs.product_key,
      COALESCE(dp.spanish_product_name, dp.english_product_name) AS product_name,
      dp.spanish_product_category_name,
      dp.spanish_product_subcategory_name,
      EXTRACT(MONTH FROM fs.order_date) AS month_in_quarter,
      fs.order_quantity
    FROM
      sales.fact_sales fs
      JOIN sales.dim_product dp ON fs.product_key = dp.product_key
    WHERE
      fs.sale_source = 'reseller_sales'
      AND dp.spanish_product_category_name <> 'Accesorio'
      AND fs.order_date >= date_trunc('quarter', CURRENT_DATE - interval '3 months')
      AND fs.order_date < date_trunc('quarter', CURRENT_D

In [15]:
llm = AzureChatOpenAI(model='gpt-4o-mini', temperature=0.25)

SYSTEM_PROMPT = """
You are a query generator that recibes an user input.
Use the following examples to create a valid Postgres SQL query that responds the user query.

User input:
<user_query>
{user_query}
</user_query

Examples:
<examples>
{retrieval_documents}
</examples>

Output:
- Only the Postgres SQL Query in plain text. DON'T USE FORMAT IN THE OUTPUT.
"""

prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("human", "{user_query}")
])

summarizer_chain = prompt | llm | StrOutputParser()

contexto_resumido = summarizer_chain.invoke({
    'user_query': QUERY,
    'retrieval_documents': json.dumps(
        ExamplesChromaCollection.format_for_llm(examples_results),
        indent=2,
        ensure_ascii=False
    )
})

print(contexto_resumido.replace('\\n', '\n'))

SELECT
  COALESCE(dp.spanish_product_name, dp.english_product_name) AS producto,
  SUM(fs.order_quantity) AS cantidad
FROM
  sales.fact_sales fs
  JOIN sales.dim_product dp ON fs.product_key = dp.product_key
GROUP BY
  producto
ORDER BY
  cantidad DESC
LIMIT 10;
